## Загрузка данных

In [1]:
import json

In [2]:
data = []
track = []

with open("hse_MARK_fall_2017", "r") as f:
    for line in f:
        beg = line.find('{')
        track = line[:beg-1]
        s= line[beg:]
        data.append(json.loads(s))
    

## Исследование уникальности ключей и классификация записей

In [3]:
# Получение уникальных ключей записей
dk = ['|'.join(d.keys()) for d in data]

In [4]:
set(dk)

{'username|context|event_source|name|ip|agent|event|host|session|referer|accept_language|time|page|event_type',
 'username|event_source|name|accept_language|time|agent|page|host|session|referer|context|ip|event|event_type',
 'username|event_type|ip|agent|host|referer|accept_language|event|event_source|context|time|page'}

In [5]:
a = 'username|context|event_source|name|ip|agent|event|host|session|referer|accept_language|time|page|event_type'
b = 'username|event_source|name|accept_language|time|agent|page|host|session|referer|context|ip|event|event_type'
c = 'username|event_type|ip|agent|host|referer|accept_language|event|event_source|context|time|page'
len(list(filter(lambda x: x == a, dk ))), len(list(filter(lambda x: x == b, dk ))), len(list(filter(lambda x: x == c, dk )))

(26700, 407752, 261961)

In [6]:
# Отметка записей уникальными ключами
for row in data:
    row['row_type'] = list(row.keys())[1]

In [7]:
# Отбираем только записи типа event_source
source_data = list(filter(lambda x: x['row_type'] == 'event_source', data))

## Наполнение датафрейма

In [8]:
import pandas as pd

In [9]:
import re

### Для видео

In [10]:
# Поля для датафрейма
source_data1 = [{'user_id':sd['context']['user_id'], 'user_name':sd['username'], 
                 'time':sd['time'], 
                 'event':(re.findall(r'id\":\s\"(.*)\",', sd['event']) if sd['event_type'].find('_video')>-1 else 0),
                 'currentTime':(re.findall(r'currentTime\":\s(.*)\}', sd['event']) if sd['event_type'].find('_video')>-1 else None),
                 'event_type':sd['event_type']} for sd in source_data]

In [11]:
df1 = pd.DataFrame(source_data1)

In [12]:
df1.head(7)

,currentTime,event,event_type,time,user_id,user_name
0,None,0,edx.course.enrollment.activated,2017-06-30T21:50:05.139616+00:00,NaN,
1,None,0,edx.course.enrollment.activated,2017-06-30T22:06:55.500221+00:00,NaN,
2,None,0,edx.course.enrollment.activated,2017-07-01T05:37:10.938597+00:00,NaN,
3,None,0,edx.course.enrollment.activated,2017-07-01T06:35:44.963665+00:00,NaN,
4,None,0,edx.course.enrollment.activated,2017-07-01T06:59:51.653981+00:00,NaN,
5,None,0,edx.course.enrollment.activated,2017-07-01T08:07:13.712422+00:00,NaN,
6,None,0,edx.course.enrollment.activated,2017-07-01T08:27:39.407180+00:00,NaN,


In [13]:
df1['video_id']=df1['event'].apply(lambda x: x[0] if x else 0)
df1['time_point']=df1['currentTime'].apply(lambda x: x[0] if x else 0)
df1['user_id'] = df1['user_id'].fillna(0)
df1['user_id'] = df1['user_id'].astype('int')

### Для задач

In [14]:
# Поля для датафрейма
source_data2 = [{'user_id': sd['context']['user_id'], 'user_name':sd['username'], 
                 'time': sd['time'], 
                 'event': sd['event'],
                 'problem_id':(re.findall(r'input_([a-z0-9]+)_',sd['event'][0])[0] if sd['event_type'] == 'problem_graded' else ''),
                 'event_type': sd['event_type']} for sd in source_data]

In [15]:
df2 = pd.DataFrame(source_data2)

In [16]:
df2 = df2[df2['event_type']=='problem_graded']


In [17]:
df2['user_id'] = df2['user_id'].fillna(0)
df2['user_id'] = df2['user_id'].astype('int')

## Формирование аналитик и вывод в csv

### Видео

In [18]:
# Выбираем последнюю по времени операцию пользователя с данным видео
df_gr=df1[df1['video_id']!=0].groupby(['user_id','video_id'])['time'].max().to_frame().reset_index().sort_values(['user_id', 'video_id'])
df_gr.head(7)

,user_id,video_id,time
0,794,f4654be000d84e1c8f41535b9fa58432,2017-11-27T22:59:54.639064+00:00
1,1366,b7318f3ff5c64f71910b26b43827a3e2,2017-11-06T17:13:02.115016+00:00
2,2249,27e8d4e7df81402b9a058ad2d7d68513,2017-10-29T11:06:43.682184+00:00
3,2249,372d624e554f4b67a3c0c2a454144123,2017-11-05T19:24:07.029842+00:00
4,2249,4debdbf5c0c44fa2891a18e253b98f22,2017-10-29T10:56:12.540579+00:00
5,2249,6309de5e5d6f4b2c86f2b78c2cb859e0,2017-10-29T11:00:22.704676+00:00
6,2249,83d66aeb3e6d418a8b485e8f2e1efade,2017-11-05T19:24:10.268013+00:00


In [19]:
video_res = df1.merge(df_gr, how='inner', on =['user_id', 'video_id', 'time'] )
video_res[['user_id', 'video_id', 'event_type', 'time_point', 'time']].sort_values(['user_id', 'time']).to_csv('video.csv', sep=';', index=False)


In [20]:
video_res.shape

(10236, 8)

### Задачи

#### в результате визуального анализа было установлено, что формулировка задачи и ответы пользователя в формате html находятся в поле event, в элементе списка с индексом 1:

In [21]:
df2['task_and_answer_html'] = df2['event'].apply(lambda x: x[1])

In [22]:
df2.head(5)

,event,event_type,problem_id,time,user_id,user_name,task_and_answer_html
3754,[input_e3d4f7d354cb4ed7959a445d3b80fad7_2_1=%D...,problem_graded,e3d4f7d354cb4ed7959a445d3b80fad7,2017-10-26T13:21:21.344170+00:00,112500,greymustdie,"\n\n\n\n<h3 class=""hd hd-3 problem-header"" id=..."
3814,[input_0e78cb41db3547ceb325d299b3b386f2_2_1%5B...,problem_graded,0e78cb41db3547ceb325d299b3b386f2,2017-10-26T13:23:50.319068+00:00,112500,greymustdie,"\n\n\n\n<h3 class=""hd hd-3 problem-header"" id=..."
3951,[input_0e78cb41db3547ceb325d299b3b386f2_2_1%5B...,problem_graded,0e78cb41db3547ceb325d299b3b386f2,2017-10-26T13:39:08.637308+00:00,223563,bogmorok,"\n\n\n\n<h3 class=""hd hd-3 problem-header"" id=..."
3997,[input_03fb67f7f2b84f51bf9d05a35404e121_2_1=%D...,problem_graded,03fb67f7f2b84f51bf9d05a35404e121,2017-10-26T13:46:46.507445+00:00,223563,bogmorok,"\n\n\n\n<h3 class=""hd hd-3 problem-header"" id=..."
4119,[input_03fb67f7f2b84f51bf9d05a35404e121_2_1=%D...,problem_graded,03fb67f7f2b84f51bf9d05a35404e121,2017-10-26T13:55:31.411437+00:00,275376,TatianaKravchuk,"\n\n\n\n<h3 class=""hd hd-3 problem-header"" id=..."


In [23]:
df2.shape

(4864, 7)

#### Здесь маленькая хитрость, связанная с недостатком времени - я не стал парсить поле task_and_answer_html на предмет выбора формулировок задач и ответов пользователя (checked, selected == true). Вывел все необходимые параметры в файл html для просмотра в любом браузере. Понимаю, что это предварительный этап, для дальнейшего разбора файла потребуется создание функционала, например, на базе модуля requests.

In [25]:
df2[['user_id', 'problem_id', 'time', 'task_and_answer_html']].sort_values(['user_id', 'time']).to_csv('problems.html', sep=';', line_terminator=f'<br/>{"-"*150}<br/>', index=False)